# Diagrama Conceitual

entidades e seus relacionamentos:

* Entidades e Atributos:

* * Municipios: Representa municípios, com os atributos:

* * * MunicipioID (Identificador único do município)

* * * CodigoMunicipioSIAFI (Código SIAFI do município)

* * * NomeMunicipioSIAFI (Nome do município)

* * * UF (Unidade Federativa do município)


* * Beneficiarios: Representa os beneficiários dos saques e pagamentos, com os atributos:

* * * BeneficiarioID (Identificador único do beneficiário)

* * * CPFBeneficiario (CPF do beneficiário)

* * * NISBeneficiario (NIS do beneficiário)

* * * NomeBeneficiario (Nome do beneficiário)


* * Saques: Representa os saques realizados, com os atributos:

* * * SaqueID (Identificador único do saque)

* * * DataSaque (Data em que o saque foi realizado)

* * * ValorParcela (Valor da parcela sacada)

* * * AnoReferencia, MesReferencia, AnoCompetencia, MesCompetencia (Datas de referência e competência)


* * Pagamentos: Representa os pagamentos efetuados.

* * * PagamentoID (Identificador único do saque)

* * * DataSaque (Data em que o saque foi realizado)

* * * ValorParcela (Valor da parcela sacada)

* * * AnoReferencia, MesReferencia, AnoCompetencia, MesCompetencia (Datas de referência e competência)



* Relacionamentos:

* * Saques e Pagamentos estão relacionados a Municipios e Beneficiarios.
* * Cada saque ou pagamento está associado a um único município e um único beneficiário.

# Diagrama Lógico

O diagrama lógico inclui chaves primárias e estrangeiras, mostrando as relações exatas entre as tabelas:

* Municipios:

* * MunicipioID: chave primária (PK)
* * CodigoMunicipioSIAFI, NomeMunicipioSIAFI, UF: atributos

* Beneficiarios:

* * BeneficiarioID: chave primária (PK)
* * CPFBeneficiario, NISBeneficiario, NomeBeneficiario: atributos

* Saques:

* * SaqueID: chave primária (PK)
* * BeneficiarioID, MunicipioID: chaves estrangeiras (FK) referenciando Beneficiarios e Municipios, respectivamente
* * DataSaque, ValorParcela, AnoReferencia, MesReferencia, AnoCompetencia, MesCompetencia: atributos

* Pagamentos:

* * PagamentoID: chave primária (PK)
* * BeneficiarioID, MunicipioID: chaves estrangeiras (FK) referenciando Beneficiarios e Municipios, respectivamente
* * ValorParcela, AnoReferencia, MesReferencia, AnoCompetencia, MesCompetencia: atributos

In [ ]:
#COmandos SQL para alteração das tabelas no banco atual 
# criando novas tabelas

CREATE TABLE Municipios (
    MunicipioID INT AUTO_INCREMENT PRIMARY KEY,
    CodigoMunicipioSIAFI INT,
    NomeMunicipioSIAFI TEXT,
    UF TEXT
);

CREATE TABLE Beneficiarios (
    BeneficiarioID INT AUTO_INCREMENT PRIMARY KEY,
    CPFBeneficiario TEXT,
    NISBeneficiario TEXT,
    NomeBeneficiario TEXT
);


# Preenchendo as novas tabelas 


INSERT INTO Municipios (CodigoMunicipioSIAFI, NomeMunicipioSIAFI, UF)
SELECT DISTINCT CodigoMunicipioSIAFI, NomeMunicipioSIAFI, UF FROM Saques
UNION
SELECT DISTINCT CodigoMunicipioSIAFI, NomeMunicipioSIAFI, UF FROM Pagamentos;

INSERT INTO Beneficiarios (CPFBeneficiario, NISBeneficiario, NomeBeneficiario)
SELECT DISTINCT CPFBeneficiario, NISBeneficiario, NomeBeneficiario FROM Saques
UNION
SELECT DISTINCT CPFBeneficiario, NISBeneficiario, NomeBeneficiario FROM Pagamentos;


# Adicionando novas colunas as tabelas Saques e Pagamentos

ALTER TABLE Saques
ADD COLUMN MunicipioID INT,
ADD COLUMN BeneficiarioID INT,
ADD FOREIGN KEY (MunicipioID) REFERENCES Municipios(MunicipioID),
ADD FOREIGN KEY (BeneficiarioID) REFERENCES Beneficiarios(BeneficiarioID);

ALTER TABLE Pagamentos
ADD COLUMN MunicipioID INT,
ADD COLUMN BeneficiarioID INT,
ADD FOREIGN KEY (MunicipioID) REFERENCES Municipios(MunicipioID),
ADD FOREIGN KEY (BeneficiarioID) REFERENCES Beneficiarios(BeneficiarioID);


# Atualizando as tabelas modificadas 

UPDATE Saques
SET 
    MunicipioID = (SELECT MunicipioID FROM Municipios WHERE Saques.CodigoMunicipioSIAFI = Municipios.CodigoMunicipioSIAFI),
    BeneficiarioID = (SELECT BeneficiarioID FROM Beneficiarios WHERE Saques.CPFBeneficiario = Beneficiarios.CPFBeneficiario);

UPDATE Pagamentos
SET 
    MunicipioID = (SELECT MunicipioID FROM Municipios WHERE Pagamentos.CodigoMunicipioSIAFI = Municipios.CodigoMunicipioSIAFI),
    BeneficiarioID = (SELECT BeneficiarioID FROM Beneficiarios WHERE Pagamentos.CPFBeneficiario = Beneficiarios.CPFBeneficiario);


# Removendo colunas desnecessárias

ALTER TABLE Saques
DROP COLUMN UF,
DROP COLUMN CodigoMunicipioSIAFI,
DROP COLUMN NomeMunicipioSIAFI,
DROP COLUMN CPFBeneficiario,
DROP COLUMN NISBeneficiario,
DROP COLUMN NomeBeneficiario;

ALTER TABLE Pagamentos
DROP COLUMN UF,
DROP COLUMN CodigoMunicipioSIAFI,
DROP COLUMN NomeMunicipioSIAFI,
DROP COLUMN CPFBeneficiario,
DROP COLUMN NISBeneficiario,
DROP COLUMN NomeBeneficiario;




